In [4]:
# train_cpu.ipynb

# 1. Importing required libraries
import os
import torch
from torch.utils import data
from networks_unetpp.Networks import UNetPlusPlus 
import torch.optim as optim
import torch.nn as nn
import time
from dataset.landslide_dataset import LandslideDataSet
import numpy as np
import matplotlib.pyplot as plt

# Define the eval_image function
def eval_image(pred, label, num_classes):
    TP = np.zeros((num_classes, 1))
    FP = np.zeros((num_classes, 1))
    TN = np.zeros((num_classes, 1))
    FN = np.zeros((num_classes, 1))
    n_valid_sample = 0

    for i in range(num_classes):
        TP[i] = np.sum((pred == i) & (label == i))
        FP[i] = np.sum((pred == i) & (label != i))
        TN[i] = np.sum((pred != i) & (label != i))
        FN[i] = np.sum((pred != i) & (label == i))
        n_valid_sample += np.sum(label == i)

    return TP, FP, TN, FN, n_valid_sample

# 2. Defining the necessary configurations and settings
data_dir = './dataset/'  # Directory where your dataset is located
train_list = './dataset/train.txt'  # Path to the train.txt file
test_list = './dataset/test.txt'  # Path to the test.txt file
input_size = '128,128'  # Input size for the images
num_classes = 2  # Number of classes (Landslide, Non-Landslide)
batch_size = 32  # Batch size for training
num_workers = 4  # Number of workers for data loading
learning_rate = 1e-3  # Learning rate
num_steps = 400  # Number of training steps
num_steps_stop = 400  # Number of training steps for early stopping
weight_decay = 5e-4  # Weight decay for regularization
snapshot_dir = './snapshots/'  # Directory to save model snapshots

# Ensure that the snapshot directory exists
if not os.path.exists(snapshot_dir):
    os.makedirs(snapshot_dir)

# 3. Loading the data
def load_data():
    src_loader = data.DataLoader(
        LandslideDataSet(data_dir, train_list, max_iters=num_steps_stop * batch_size, set='labeled'),
        batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)

    test_loader = data.DataLoader(
        LandslideDataSet(data_dir, test_list, set='unlabeled'),
        batch_size=1, shuffle=False, num_workers=num_workers, pin_memory=True)

    return src_loader, test_loader

src_loader, test_loader = load_data()

# 4. Initializing the U-Net model
model = UNetPlusPlus(n_classes=num_classes)

# 5. Defining the optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
cross_entropy_loss = nn.CrossEntropyLoss(ignore_index=255)


Number of files loaded: 16599
Number of files loaded: 800


In [5]:

# 6. Training function
def train_model(num_steps_stop):
    hist = np.zeros((num_steps_stop, 3))
    for batch_id, src_data in enumerate(src_loader):
        if batch_id == num_steps_stop:
            break

        start_time = time.time()
        model.train()
        optimizer.zero_grad()

        images, labels, _, _ = src_data

        # Forward pass
        pred = model(images)

        # Compute loss
        labels = labels.long()
        loss = cross_entropy_loss(pred, labels)

        # Backpropagation and optimization
        loss.backward()
        optimizer.step()

        # Calculate batch accuracy
        _, predicted_labels = torch.max(pred, 1)
        predicted_labels = predicted_labels.detach().cpu().numpy()
        labels = labels.numpy()

        batch_oa = np.sum(predicted_labels == labels) * 1.0 / len(labels.reshape(-1))
        hist[batch_id, 0] = loss.item()
        hist[batch_id, 1] = batch_oa
        hist[batch_id, -1] = time.time() - start_time

        # Print progress
        if (batch_id + 1) % 10 == 0:
            print(f'Iter {batch_id+1}/{num_steps} Time: {np.mean(hist[batch_id-9:batch_id+1,-1]):.2f} Batch_OA = {np.mean(hist[batch_id-9:batch_id+1,1])*100:.1f} cross_entropy_loss = {np.mean(hist[batch_id-9:batch_id+1,0]):.3f}')


In [7]:

# 8. Run training and testing
train_model(num_steps_stop)

Iter 10/400 Time: 46.18 Batch_OA = 83.5 cross_entropy_loss = 0.493
Iter 20/400 Time: 43.79 Batch_OA = 98.1 cross_entropy_loss = 0.270
Iter 30/400 Time: 53.94 Batch_OA = 98.0 cross_entropy_loss = 0.202
Iter 40/400 Time: 55.33 Batch_OA = 98.3 cross_entropy_loss = 0.151
Iter 50/400 Time: 48.26 Batch_OA = 98.6 cross_entropy_loss = 0.120
Iter 60/400 Time: 45.77 Batch_OA = 98.4 cross_entropy_loss = 0.101
Iter 70/400 Time: 47.68 Batch_OA = 97.9 cross_entropy_loss = 0.093
Iter 80/400 Time: 47.17 Batch_OA = 98.6 cross_entropy_loss = 0.074
Iter 90/400 Time: 47.47 Batch_OA = 98.3 cross_entropy_loss = 0.071
Iter 100/400 Time: 47.16 Batch_OA = 98.2 cross_entropy_loss = 0.070
Iter 110/400 Time: 47.16 Batch_OA = 98.6 cross_entropy_loss = 0.058
Iter 120/400 Time: 47.08 Batch_OA = 98.4 cross_entropy_loss = 0.058
Iter 130/400 Time: 47.09 Batch_OA = 98.3 cross_entropy_loss = 0.058
Iter 140/400 Time: 47.17 Batch_OA = 98.5 cross_entropy_loss = 0.050
Iter 150/400 Time: 47.08 Batch_OA = 98.8 cross_entropy_lo

In [8]:
# 7. Saving the Trained Model
def save_model(model, snapshot_dir, model_name='unet_plus_plus_trained_model1.pth'):
    # Ensure the snapshot directory exists
    if not os.path.exists(snapshot_dir):
        os.makedirs(snapshot_dir)
    
    # Save the model's state_dict (recommended way of saving models)
    model_path = os.path.join(snapshot_dir, model_name)
    torch.save(model.state_dict(), model_path)
    print(f'Model saved to {model_path}')

# Example usage after your training loop:
save_model(model, snapshot_dir, 'unet_plus_plus_trained_model1.pth')

Model saved to ./snapshots/unet_plus_plus_trained_model1.pth
